Volgende stappen
- los probleem met JSON serializable op: SegmentedText is niet JSON serializable. DONE: subclass JSONEncoder.
- maak Flask based service, die weer asearch functies aanroept - DONE
- schrijf handler functies voor aservice REST calls, in de aservice package, op basis huidige functies

In [1]:
import sys
sys.path.append('../packages')

from textservice import segmentedtext

In [5]:
import lxml
from lxml import etree
import uuid

path = '../data/input/tei-samples/bosb002graa04_01.xml'
datadir = '../data/output/'

resource_id = 'bosb002graa04_01'
text_store = 'tei_textstore.json'
annotation_store = 'tei_annotationstore.json'

_last_page_begin_index = 0
_last_section_begin_index = -1
_last_chapter_begin_index = -1
_last_paragraph_begin_index = -1
_last_head_begin_index = -1

_last_page_end_index = -1
_last_section_end_index = -1
_last_chapter_end_index = -1
_last_paragraph_end_index = -1
_last_head_end_index = -1

_last_page_id = ""

def get_file_sequence_for_container(text_container):
    return [path]

def get_root_tree_element(file):
    # use iterparse to traverse the xml hierarchy, depth first, post order
    return etree.iterparse(path, events=('start','end'))

# handle each of the elements in the hierarchy according to 'layer type'
def handle_element(action,e,text,annotations):  
    global resource_id
    global _last_page_begin_index
    global _last_section_begin_index
    global _last_chapter_begin_index
    global _last_paragraph_begin_index
    global _last_head_begin_index

    global _last_page_end_index
    global _last_section_end_index
    global _last_chapter_end_index
    global _last_paragraph_end_index
    global _last_head_end_index
    
    global _last_page_id

    if action == 'start':
        # store last begin indexes
        if e.tag == 'p':
            _last_paragraph_begin_index = text.len()               
        elif e.tag == 'div' and e.get('type') == 'chapter':
            _last_chapter_begin_index = text.len()
        elif e.tag == 'div' and e.get('type') == 'section':
            _last_section_begin_index = text.len()
        elif e.tag == 'head':
            _last_head_begin_index = text.len()
    elif action == 'end':
        if e.tag == 'p': 
            # leaf text element, add to all_textelements, also include text after possible pb's
            for index, t in enumerate(e.itertext()):
                text.append(t.strip())
                if index > 0: # assume: caused by pb contained within p. Update page end.
                    _last_page_end_index = text.len()-1
            
            _last_paragraph_end_index = text.len()-1

            if _last_paragraph_begin_index <= _last_paragraph_end_index:
                annotations.append({'resource_id': resource_id, 'label':'paragraph','begin_anchor': text._anchors[_last_paragraph_begin_index],\
                            'end_anchor':text._anchors[_last_paragraph_end_index],'id': 'annot_'+str(uuid.uuid4())})
        elif e.tag == 'head':
            # leaf text element, add to all_textelements
            text.append(e.text)
            
            _last_head_end_index = text.len()-1
            annotations.append({'resource_id': resource_id, 'label':'head','begin_anchor': text._anchors[_last_head_begin_index],\
                            'end_anchor':text._anchors[_last_head_end_index],'id': 'annot_'+str(uuid.uuid4())}) 
        elif e.tag == 'div' and e.get('type') == 'chapter':
            _last_chapter_end_index = text.len()-1
            annotations.append({'resource_id': resource_id, 'label':'chapter','begin_anchor': text._anchors[_last_chapter_begin_index],\
                            'end_anchor':text._anchors[_last_chapter_end_index],'id': 'annot_'+str(uuid.uuid4())})            
        elif e.tag == 'div' and e.get('type') == 'section':
            _last_section_end_index = text.len()-1
            annotations.append({'resource_id': resource_id, 'label':'section','begin_anchor': text._anchors[_last_section_begin_index],\
                            'end_anchor':text._anchors[_last_section_end_index],'id': 'annot_'+str(uuid.uuid4())})               
        elif e.tag == 'pb':
            # first store the 'previous' page, then store begin and end of currently closed page
            annotations.append({'resource_id': resource_id, 'label':'page','begin_anchor': text._anchors[_last_page_begin_index],\
                            'end_anchor':text._anchors[_last_page_end_index],'id': _last_page_id}) 
            _last_page_begin_index = _last_page_end_index
            _last_page_end_index = text.len()-1 
            _last_page_id = f"page-{e.get('n')}"
            
    return        

def traverse(node,node_label,text,annotations):
    for action, elem in node:
        handle_element(action,elem,text,annotations)  
    return
    
# Process per file, properly concatenate results, maintaining proper referencing the baseline text elements
resource_id = 'bosb002graa04_01'
all_textelements=segmentedtext.SplittableSegmentedText(resource_id)

all_annotations=[]

for f_name in get_file_sequence_for_container(resource_id):
    text_segments = segmentedtext.SplittableSegmentedText()
    annotation_array = []
            
    source_data = get_root_tree_element(f_name)

    traverse(source_data,'',text_segments,annotation_array)
           
    # properly concatenate annotation info taking ongoing line indexes into account - trivial, do not apply in this case
#    for ai in annotation_array:
#        ai['begin_index'] += len(all_textlines)
#        ai['end_index'] += len(all_textlines)
    
    all_textelements.extend(text_segments)       
    all_annotations.extend(annotation_array)
    

In [6]:
all_textelements.len()

3130

In [7]:
# return generator for annotations with a specific label
#def get_annotations_of_type(type,annotations):
#    return (a for a in annotations if a['label'] == type)

from annotation import asearch

In [8]:
sample_para = [paras for paras in asearch.get_annotations_of_type('paragraph', all_annotations)][103]
print(sample_para['begin_anchor'])
print(sample_para['end_anchor'])

anchor_d3de613e-1155-4404-835b-b080416addbc
anchor_d3de613e-1155-4404-835b-b080416addbc


In [9]:
all_textelements.slice(sample_para['begin_anchor'], sample_para['end_anchor'])

['‘Die goede Geertrui!’ zeide Elisabeth glimlachend tot Courtenay; ‘zij is zoo ingenomen met den titel, dien mijn vader haar gaf, toen zij mijne min werd, dat zij zelfs hare gehechtheid aan mij onder de stijve windsels der étiquette verbergt. Maar ik schaam mij, ik ben wel egoïstisch, beste Graaf! ik heb er tot hiertoe slechts aan gedacht u mijne eigene grieven te verhalen, en was daarna zoo verdiept in het luisteren naar u, dat ik er bij vergat, hoe vermoeid gij moet zijn en hoe verlangend naar eenige verkwikking.’']

In [10]:
begin_anchor = all_textelements.split(sample_para['begin_anchor'], 11)
end_anchor = all_textelements.split(begin_anchor, 8)

In [11]:
all_textelements.slice(begin_anchor, begin_anchor)

['Geertrui']

In [12]:
all_annotations.append({'resource_id': resource_id, 'label':'entity','begin_anchor': begin_anchor,'end_anchor': begin_anchor,\
                    'id': 'annot_'+str(uuid.uuid4()), 'entity_type': 'per', 'entity_text': 'Geertrui'})

In [13]:
entities = [ents for ents in asearch.get_annotations_of_type('entity', all_annotations)]
entities

[{'resource_id': 'bosb002graa04_01',
  'label': 'entity',
  'begin_anchor': anchor_5318f4e3-24ac-43a8-84b3-041a12a07e6f,
  'end_anchor': anchor_5318f4e3-24ac-43a8-84b3-041a12a07e6f,
  'id': 'annot_13044244-89c0-4dd2-9755-4a5fdbb1fea7',
  'entity_type': 'per',
  'entity_text': 'Geertrui'}]

In [15]:
# print all annotations overlapping with the entity
for a in asearch.get_annotations_overlapping_with(entities[0]['begin_anchor'],entities[0]['end_anchor'],\
            all_annotations,resource_id):
    print(a)

{'resource_id': 'bosb002graa04_01', 'label': 'page', 'begin_anchor': anchor_959ec341-c694-4f7b-b040-258bfc191f2d, 'end_anchor': anchor_f1c2274d-1f87-4985-94d9-672118b03d41, 'id': 'page-22'}
{'resource_id': 'bosb002graa04_01', 'label': 'section', 'begin_anchor': anchor_dfc00674-ee82-492d-af99-2a44190b9155, 'end_anchor': anchor_15650e41-aeac-4bb1-8ed3-025e4a516612, 'id': 'annot_a56ca19d-d32d-4477-bee1-12c1f2b4821b'}
{'resource_id': 'bosb002graa04_01', 'label': 'chapter', 'begin_anchor': anchor_dfc00674-ee82-492d-af99-2a44190b9155, 'end_anchor': anchor_15650e41-aeac-4bb1-8ed3-025e4a516612, 'id': 'annot_cf0e329a-d7d4-4c27-9ba0-5325ac117eb0'}
{'resource_id': 'bosb002graa04_01', 'label': 'entity', 'begin_anchor': anchor_5318f4e3-24ac-43a8-84b3-041a12a07e6f, 'end_anchor': anchor_5318f4e3-24ac-43a8-84b3-041a12a07e6f, 'id': 'annot_13044244-89c0-4dd2-9755-4a5fdbb1fea7', 'entity_type': 'per', 'entity_text': 'Geertrui'}


Gebruik bovenstaande voor het visualiseren van alle tekst voor een pagina, of alle heads (vanaf het midden van het boek)

In [16]:
for a in asearch.get_annotations_of_type('page', all_annotations):
    if a['id'] == 'page-116':
        print(a)

{'resource_id': 'bosb002graa04_01', 'label': 'page', 'begin_anchor': anchor_d29032f8-1f7a-4aef-b913-02ffe07337ef, 'end_anchor': anchor_9bb68a7a-eb7d-46a4-aef1-fb4cb4135add, 'id': 'page-116'}


In [17]:
p116 = [p for p in asearch.get_annotations_of_type('page', all_annotations) if p['id'] == 'page-116'][0]

for t in all_textelements.slice(p116['begin_anchor'],p116['end_anchor']):
    print(f"{t}\n")

dedigen. Het ware hem wellust geweest, slechts eens den zoeten naam der geliefde voor een vertrouwd oor te mogen uitspreken. En toch moest hij laaghartig en zelfzuchtig schijnen, en toch gevoelde hij dat de ander niet meer gelooven kon aan de goedheid van zijn hart, toen hij antwoordde, zooals zijn gegeven woord het eischte:

‘Neen, Arundel, neen! ik kan haar niet helpen; voor haar vermag ik niets; vooralsnog niets;... misschien in het vervolg;... wie weet.... geloof mij, mijn vriend, laat ons geduld hebben.’

‘Dus heeft de beklagenswaardige ook u verloren,’ hervatte de Groot-Kamerheer koel en met stugheid. ‘Ik moet u nog met eene vraag lastig vallen, Mylord; het is de laatste over dit onderwerp. Zult gij mij tegen zijn, zoo ik in haar voordeel spreek?’

‘Op mijn ridderwoord, neen!’ riep Devonshire schielijk; zijne oogen schitterden van vreugde over eene uitkomst, die zoozeer met zijne wenschen instemde en hij stond op en vatte Arundel bij de hand. Zoo brandend jaagde het bloed hem doo

In [18]:
import json

segtextJSONData = json.dumps(all_textelements, indent=4, cls=segmentedtext.SegmentEncoder)
print(segtextJSONData)

{
    "resource_id": "bosb002graa04_01",
    "_ordered_segments": [
        "GEBRUIKT EXEMPLAAR",
        "eigen exemplaar dbnl",
        "",
        "ALGEMENE OPMERKINGEN",
        "Dit bestand biedt, behoudens een aantal hierna te noemen ingrepen, een\n                    diplomatische weergave van",
        "De graaf van Devonshire",
        "van A.L.G. Bosboom-Toussaint in een ongedateerde herdruk van ca. 1900. De eerste druk is uit 1838. Deze uitgave bevat ook de novellen",
        "Eene familie-legende",
        "(eerste druk 1864),",
        "De Alkmaarsche wees",
        "(eerste druk 1850) en",
        "Een nacht in een armstoel",
        "(niet los verschenen, eerste druk  in verhalenbundel",
        "De Alkmaarsche wees en eenige andere novellen",
        "in 1854).",
        "",
        "REDACTIONELE INGREPEN",
        "Bij de omzetting van de gebruikte bron naar deze publicatie in de dbnl is een aantal delen van de tekst niet overgenomen. Hieronder volgen de tekstgedeelten

write a function that creates a textstore if it does not exist. Otherwise read the json from the textstore and add the IndexedSegmentedText to the _resources list

In [19]:
def add_segmented_text_to_store(segmented_text, store_name):
    try:
        with open(datadir+store_name, 'r') as filehandle:
            data = json.loads(filehandle.read())
    except FileNotFoundError:
        data = {'_resources' : []}
        
    data['_resources'].append(segmented_text)
    
    with open(datadir+store_name, 'w') as filehandle:
        json.dump(data, filehandle, cls=segmentedtext.SegmentEncoder)           

In [20]:
add_segmented_text_to_store(all_textelements, text_store)

In [21]:
def add_annotations_to_store(annotations, store_name):
    try:
        with open(datadir+store_name, 'r') as filehandle:
            data = json.loads(filehandle.read())
    except FileNotFoundError:
        data = []
        
    data.extend(annotations)
    
    with open(datadir+store_name, 'w') as filehandle:
        json.dump(data, filehandle, cls=segmentedtext.AnchorEncoder)           

In [22]:
add_annotations_to_store(all_annotations, annotation_store)